In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

import sqlite3 as dbms
import sqlalchemy
import sys


conn = dbms.connect("students.sqlite3")
cursor = conn.cursor()

with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()
    

if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()


DeclBase = declarative_base()
engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
class Program(DeclBase):
    __tablename__ = 'programs'

    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="program")

    def __init__(self, name):
        self.name = name

class Student(DeclBase):
    __tablename__ = 'students'

    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.program = program
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        
class Course(DeclBase):
    __tablename__ = 'courses'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'

    semester_number = Column(Integer, primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

In [ ]:
se_1 = Program("Теоретическая физика")
se_2 = Program("Бионанотехнологии")


st1 = Student("000001", "Пупкин", "Василий", "Алексеевич", se_1)
st2 = Student("000002", "Чеовеков", "Человек", "Человекович", se_2)
st3 = Student("000003", "Пупкин", "Иван", "Алексеевич", se_1)
st4 = Student("000004", "Бобов", "Имхотеп", "Иванович", se_2)


c1 = Course("Математический анализ")
c2 = Course("Физическая химия")
c3 = Course("Математическая биология")

pc1_1 = Program_course("1", c1, se_1)
pc1_2 = Program_course("3", c2, se_1)
pc1_3 = Program_course("4", c3, se_1)
pc2_1 = Program_course("1", c1, se_2)
pc2_2 = Program_course("2", c2, se_2)
pc2_3 = Program_course("4", c3, se_2)

m1_1 = Mark("3", st1, c1)
m1_2 = Mark("4", st1, c2)
m1_3 = Mark("5", st1, c3)

m2_1 = Mark("3", st2, c1)
m2_2 = Mark("4", st2, c2)
m2_3 = Mark("5", st2, c3)

m3_1 = Mark("4", st3, c1)
m3_2 = Mark("3", st3, c2)
m3_3 = Mark("5", st3, c3)

m4_1 = Mark("4", st4, c1)
m4_2 = Mark("3", st4, c2)
m4_3 = Mark("5", st4, c3)

session.add_all([se_1, se_2, st1, st2, st3, st4, c1, c2, pc1_1, pc1_2, pc1_3, pc2_1, pc2_2, pc2_3, m1_1, m1_2, m1_3, m2_1, m2_2, m2_3, m3_1, m3_2, m3_3, m4_1, m4_2, m4_3]) 
session.commit()

In [ ]:
print("Оценки:")
for p in session.query(Student):
    print()
    print(p.surname, p.name, "("+ p.program.name +")")
    for s in p.marks:
        print("- ", s.course.name + " -", s.mark)

print()
print("==================================")
print()

print("Список студентов:")
for p in session.query(Program):
    print()
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.surname, s.name, s.patronymic)

print()
print("==================================")
print()
        
print("Прогамма обучения и предметы:")
for p in session.query(Program):
    print()
    print("Программа: ", p.name)
    for s in p.programs_courses:
        print("- ", s.course.name, "в " + str(s.semester_number) + " семестре")
